In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

test_start_grid = np.array(
    [[0, 2, 2, 0],
     [0, 2, 2, 0],
     [0, 0, 3, 0],
     [0, 1, 0, 0],
     [0, 0, 0, 0]])

test_start_grid_2 = np.array(
    [[1, 2, 2, 3],
     [1, 2, 2, 3],
     [0, 4, 4, 0],
     [5, 6, 7, 8],
     [5, 9, 10, 8]])

test_start_grid_3 = np.array(
    [[1, 2, 2, 3],
     [1, 2, 2, 3],
     [0, 4, 4, 0],
     [5, 6, 0, 0],
     [5, 9, 10, 0]])

test_end_grid = np.array(
    [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 2, 2, 0],
     [0, 2, 2, 0]])



In [3]:
from tiles.solver import Solver

solver = Solver(start_grid=test_start_grid_2, end_grid=test_end_grid)

In [8]:
grids = solver.solve(log_frequency=20_000)

Length of grid path: 123 Num grids analysed: 1240000 Length of queue: 113933 g_cost: 122 h_cost: 0 min_h_cost: 0 
Length of grid path: 63 Num grids analysed: 1240000 Length of queue: 113932 g_cost: 62 h_cost: 3 min_h_cost: 0 
Length of grid path: 64 Num grids analysed: 1300000 Length of queue: 121490 g_cost: 63 h_cost: 3 min_h_cost: 0 
Length of grid path: 64 Num grids analysed: 1320000 Length of queue: 121505 g_cost: 63 h_cost: 3 min_h_cost: 0 
Length of grid path: 64 Num grids analysed: 1320000 Length of queue: 121504 g_cost: 63 h_cost: 3 min_h_cost: 0 
Length of grid path: 64 Num grids analysed: 1340000 Length of queue: 121763 g_cost: 63 h_cost: 3 min_h_cost: 0 
Length of grid path: 64 Num grids analysed: 1360000 Length of queue: 122431 g_cost: 63 h_cost: 3 min_h_cost: 0 
Length of grid path: 104 Num grids analysed: 1380000 Length of queue: 123275 g_cost: 103 h_cost: 1 min_h_cost: 0 
Length of grid path: 112 Num grids analysed: 1420000 Length of queue: 127724 g_cost: 111 h_cost: 0 m

In [5]:
grid_hash = None
min_v = float('inf')
for h, v in solver.hash_to_h_cost.items():
    if v < min_v:
        grid_hash = h
        min_v = v
print(min_v)

0


In [12]:
grid_hash = None
min_v = float('inf')
for h, v in solver.hash_to_h_cost.items():
    if v > 0:
        continue
    v = solver.hash_to_g_cost[h]
    if v < min_v:
        grid_hash = h
        min_v = v
print(min_v)

86


In [21]:
from tiles.solver import get_distance_to_position
import functools
heuristic_function = functools.partial(
    get_distance_to_position,
    position_x=solver.end_x,
    position_y=solver.end_y,
    target_block_value=solver.target_block_value,
)
import tqdm

for h, grid in tqdm.tqdm(solver.hash_to_grid.items()):
    if heuristic_function(grid) == 0:
        print(h)

100%|████████████████████████████████████████████████████████| 1892701/1892701 [00:20<00:00, 93093.82it/s]

e3c30a483613ab4
26145647daaa12f


In [38]:
grid_hash1 = 'e3c30a483613ab4'
# grid_hash2 = '26145647daaa12f'

In [14]:
# grid_hash = solver.array_hash_function(grids[-1])
# grid_hash = solver.queue[0].grid_hash
print(solver.hash_to_g_cost[grid_hash])
print(solver.hash_to_h_cost[grid_hash])

# solver._pop_from_queue()

86
0


In [39]:
from tiles.solver import get_grid_path_from_hash
grids = get_grid_path_from_hash(
    grid_hash=grid_hash, hash_to_parent_hash=solver.hash_to_parent_hash, hash_to_grid=solver.hash_to_grid
)

In [35]:
grids[-1]

array([[ 1,  5,  7,  6],
       [ 1,  5,  8,  3],
       [ 4,  4,  8,  3],
       [ 9,  2,  2,  0],
       [10,  2,  2,  0]])

In [45]:
from bokeh.plotting import show
from bokeh.io import output_notebook

import numpy as np

from tiles.view import modify_document_factory

modify_document = modify_document_factory(grids=grids)

output_notebook()
show(modify_document) # If running this on a remote server, add the url to the Jupyter server with notebook_url='{IP}:{port}'

Loading BokehJS ...

In [40]:
grid_array = np.stack(grids, axis=-1)

In [41]:
grid_array.shape

(5, 4, 119)

In [42]:
np.save('solution2.npy', grid_array)